In [1]:
import pandas as pd
import itertools
import math

from pandas import concat

from ipynb.fs.full.Utils              import *
from ipynb.fs.full.Values             import *
from ipynb.fs.full.ReturnValue        import *

In [ ]:
def deleteDocuments(data, index, result, args):
    ret = ZERO

    return ret

In [ ]:
def deleteSentences(data, index, result, args):
    ret = ZERO

    return ret

In [ ]:
def deleteNaN(data, index, result, args):
    ret = ZERO

    return ret

In [ ]:
def deleteCharacters(data, index, result, args):
    ret = ZERO

    return ret

In [ ]:
def deleteWords(data, index, result, args):
    ret = ZERO

    return ret

In [ ]:
def adjustBIOFormat(data, index, result, args):
    ret = ZERO

    return ret

In [ ]:
def replaceBIOFormat(data, index, result, args):
    ret = ZERO

    return ret

In [2]:
def deleteDocs(data, docIDs):
    ret = ZERO
    
    if (data is not None and 
        data.data is not None and 
        data.documentIDColumn is not None and 
        docIDs is not None):
        
        Logger.printInfo(deletingDocuments.format(len(docIDs)))

        if len(docIDs) > ZERO:
            
            delDoc = ZERO
            
            for docID in docIDs:
                dataFilter = data.getData()[data.documentIDColumn] == docID
                data.data = data.data[~dataFilter].reset_index(drop = True)
                delDoc += sum(dataFilter)
                
            Logger.printInfo(deletedAmountDocuments.format(delDoc))
    else:
        Logger.printWarning(deletingDocumentsNotPossible)
        ret = ONE
        
    return ret

In [3]:
def deleteSentences(data, sentIDs):
    ret = ZERO
    
    if (data is not None and 
        data.data is not None and 
        data.sentenceIDColumn is not None and 
        sentIDs is not None):
        
        Logger.printInfo(deletingSentences.format(len(sentIDs)))
        
        if len(sentIDs) > ZERO:
            
            delSent = ZERO
            
            for sentID in sentIDs:
                dataFilter = data.data[data.sentenceIDColumn] == sentID
                data.data = data.data[~dataFilter].reset_index(drop = True)
                delSent += sum(dataFilter)
                
            Logger.printInfo(deletedAmountDocuments.format(delSent))
    else:
        Logger.printWarning(deletingSentencesNotPossible)
        ret = ONE
        
    return ret

In [4]:
def deleteNaN(data, delDocs = False, delSent = False):
    ret = ZERO
    
    if (data is not None and 
        data.data is not None and 
        data.wordColumn is not None and 
        data.wordIDColumn is not None):
        
        dataFilter = pd.isnull(data.data)
        dataFilter = dataFilter[data.wordColumn]
        
        Logger.printInfo(deletingNaNWords.format(sum(dataFilter)))
        
        docIDs = []
        if delDocs and data.documentIDColumn is not None:
            docIDs = data.data[dataFilter][data.documentIDColumn]
        else:
            if delDocs:
                Logger.printWarning(deletingConnectedDocumentsNotPossible)
                ret = TWO
                
        sentIDs = []
        if delSent and data.sentenceIDColumn is not None:
            sentIDs = data.data[dataFilter][data.sentenceIDColumn]
        else:
            if delSent:
                Logger.printWarning(deletingConnectedSentencesNotPossible)
                ret = THREE
        
        data.data = data.data[~dataFilter].reset_index(drop=True)
        data.data[data.wordIDColumn] = range(ONE, len(data.data) + ONE)
        
        Logger.printInfo(deletedAmountNaN.format(sum(dataFilter)))
        
        if (len(docIDs) > ZERO):
            Logger.printInfo(deletingConnectedDocuments)
            deleteDocs(data, docIDs)
            
        if (len(sentIDs) > ZERO):
            Logger.printInfo(deletingConnectedSentences)
            deleteSentences(data, sentIDs)
    else:
        Logger.printWarning(deletingNaNWordsNotPossible)
        ret = ONE
    
    return ret

In [5]:
def deleteWords(data, string, delDocs = False, delSent = False):
    ret = ZERO
    
    if (data is not None and 
        data.data is not None and 
        string is not None and 
        data.wordColumn is not None and 
        data.wordIDColumn is not None):
        
        dataFilter = data.data[data.wordColumn] == string
        
        Logger.printInfo(deletingWords.format(sum(dataFilter), string))
        
        docIDs = []
        if delDocs and data.documentIDColumn is not None:
            docIDs = data.data[dataFilter][data.documentIDColumn]
        else:
            if delDocs:
                Logger.printWarning(deletingConnectedDocumentsNotPossible)
                ret = TWO
                
        sentIDs = []
        if delSent and data.sentenceIDColumn is not None:
            sentIDs = data.data[dataFilter][data.sentenceIDColumn]
        else:
            if delSent:
                Logger.printWarning(deletingConnectedSentencesNotPossible)
                ret = THREE
        
        data.data = data.data[~dataFilter].reset_index(drop=True)
        data.data[data.wordIDColumn] = range(ONE, len(data.data) + ONE)
        
        Logger.printInfo(deletedAmountWords.format(sum(dataFilter), string))

        if (len(docIDs) > ZERO):
            Logger.printInfo(deletingConnectedDocuments)
            deleteDocs(data, docIDs)
            
        if (len(sentIDs) > ZERO):
            Logger.printInfo(deletingConnectedSentences)            
            deleteSentences(data, sentIDs)
    else:
        Logger.printWarning(deletingWordsNotPossible)
        ret = ONE
    
    return ret

In [6]:
"""
This function splits the word in <data.getData()> on index <rowIndex> at every 
encounter of <splitString>. The result is a data frame which has the same 
columns as <data.getData()> having the splits of the word and the <splitString> as 
single rows in the column <data.wordColumn> saved in results.

Example:

The string "Low-and-Middle-Income" if splitted at the character "-" will result
in a data frame with seven rows containing "Low", "-", "and", "-", "Middle", 
"-", and "Income" as words. Every other entry of that row will be copied.

The created data frame will be stored in the given parameter <results> at 
<rowIndex>, therefore <results> needs to have the same size as <data.getData()>.
Additionally, the data in <results> is always copy of the data.  

data: an object of the type Data where the words are stored.
results: a list with the same length as <data.getData()> where the results are
    stored. The element contained in this list on the given index is 
    overwritten.
rowIndex: an index between zero an the length of <data.getData()>. The word which 
    will be splitted is located on this index.
splitString: the string where the word will be splitted. 
return: zero if everything went well,
    one if one of the parameters or needed variables in the parameters is None,
    two, if <rowIndex> is negative, larger than the length of <data.getData()>, or
        <result> and <data.getData()> have not the same length.
    three, if the row is None,
    four, if the word is None
"""
def splitAtIndex(data, index, resultData, resultValue, args) -> ReturnValue:
    ret = ReturnValue(ZERO)
    resultValue[index] = -ONE

    # Check if the main parameters are all given.
    if (data is not None and data.getData() is not None and 
        data.getWordColumn() is not None and args is not None):

        if (len(data.getData()) > index and index >= ZERO):

            columns = data.getData().columns
            row = data.getData().iloc[index]
            row = (pd.DataFrame(data = [row], columns = columns).
                reset_index(drop = True))
            columns = row.columns

            if (row is not None):

                word = str(row[data.getWordColumn()][ZERO])
                splitString = args[ZERO]

                # Check if there is a string which could be splitted.
                if (word is not None and splitString is not None):

                    # If the string is shorter or has the same length as 
                    # <splitString>, there is nothing to do and the row is just
                    # copied.
                    if (len(word) > len(splitString)):
                        splits = word.split(splitString)
                        count = len(splits)

                        # If there is more than just one split, an extra data
                        # frame needs to be created and filled with the
                        # data.
                        if (count > ONE):

                            # Doubling the splits.
                            splits = list(itertools.chain.from_iterable(
                                zip(splits, splits)))
                            
                            count = count * TWO
                            
                            # Replace every second element with the 
                            # <splitString>.
                            splits[ONE::TWO] = (splitString * 
                                math.floor(count / TWO))
                            
                            # Remove the last <splitString>, since it was never
                            # there.
                            splits = splits[ZERO:count - ONE]

                            count = count - ONE

                            # Create the new data frame containing the same
                            # data as <row>. 
                            resultData[index] = pd.DataFrame(
                                data = [row.iloc[ZERO]] * count, 
                                columns = columns).reset_index(drop = True)
                            
                            # Replacing the strings in the column
                            # <data.wordColumn> with the splits.
                            resultData[index][data.getWordColumn()] = splits
                            resultValue[index] = ONE
                        else:
                            resultData[index] = row
                            resultValue[index] = ZERO
                    else:
                        resultData[index] = row
                        resultValue[index] = ZERO
                else:
                    ret.setValue(FOUR)
            else:
                ret.setValue(THREE)
        else:
            ret.setValue(TWO)
    else:
        ret.setValue(ONE)

    return ret

In [8]:
def adjustBIOFormat(data, bTag, iTag, oTag):
    ret = ZERO

    if (data is not None and 
        data.data is not None and 
        bTag is not None and
        len(bTag) > ZERO and
        iTag is not None and
        len(iTag) > ZERO and
        oTag is not None and
        len(oTag) > ZERO and
        data.labelColumn is not None):

        Logger.printInfo(adjustingBIOFormat)
        
        # Zero: O tag was the last
        # One:  I tag was the last
        # TWO:  B tag was the last
        state = ZERO
        
        length = len(data.data)
        now = datetime.now()
        
        corrected = ZERO
        
        for index in range(ZERO, length):
            if state == ZERO:
                if data.data.loc[index, data.labelColumn] == bTag:
                    state = TWO
                else:
                    # Need to correct
                    if data.data.loc[index, data.labelColumn] == iTag:
                        data.data.loc[index, data.labelColumn] = bTag
                        corrected += ONE
                        state = TWO
            else:
                if state == ONE:
                    # Need to correct
                    if data.data.loc[index, data.labelColumn] == bTag:
                        data.data.loc[index, data.labelColumn] = iTag
                        corrected += ONE
                    else:
                        if data.data.loc[index, data.labelColumn] == oTag:
                            state = ZERO
                else:
                    # Need to correct
                    if data.data.loc[index, data.labelColumn] == bTag:
                        data.data.loc[index, data.labelColumn] = iTag
                        corrected += ONE
                        state = ONE
                    else:
                        if data.data.loc[index, data.labelColumn] == iTag:
                            state = ONE
                        else:
                            state = ZERO
            Logger.printProgress(index + ONE, length, now)
        
        Logger.printInfo(adjustedBIOFormat.format(corrected))
    else:
        Logger.printWarning(adjustingBIOFormatNotPossible)
        ret = ONE

    return ret

In [9]:
"""
TODO: methode ist nicht sonderlich gut: nicht existente sentence ids werden trotzdem verwendet
"""
def createSentences(data, noPreSpaceCharacters, noPostSpaceCharacters):
    ret = None

    if (data.sentenceIDColumn is not None and data.wordColumn is not None):
        
        sentenceIDs = data.data[data.sentenceIDColumn].unique()
        sentences = [EMPTYSTRING] * len(sentenceIDs)

        now = datetime.now()

        nopostspace = False
        
        for sentenceIndex, sentenceID in enumerate(sentenceIDs):
            sentenceWords = data.data[data.data[data.sentenceIDColumn] == sentenceID]
            sentenceWords = sentenceWords[data.wordColumn]
            
            sentences[sentenceIndex] = concat(sentenceWords, noPreSpaceCharacters, noPostSpaceCharacters)

            Logger.printProgress(sentenceIndex + ONE, len(sentences), now)

        ret = sentences

    return ret

In [10]:
def diffWord(data, words, prints = FIVE):
    ret = ZERO

    if data is not None and words is not None and data.data is not None and data.wordColumn is not None:
            
        now = datetime.now()
        length = len(data.data)
        for index in range(ZERO, length):
            
            row = data.data.loc[index]

            if index < len(words) and row[data.wordColumn] != words[index]:

                

                Logger.printInfo("Difference found.")
                fromIndex = index - prints
                if (fromIndex < 0):
                    fromIndex = 0
                toIndex = index + prints
                if (toIndex > len(data.data)):
                    toIndex = len(data.data)

                for printIndex in range(fromIndex, toIndex):
                    if printIndex == index:
                        Logger.printInfo(printIndex)
                    Logger.printInfo(diff.format(data.data[data.wordColumn][printIndex], words[printIndex]))
                break

            Logger.printProgress(index + ONE, length, now)
    else:
        Logger.printWarning(diffWordsNotPossible)
        ret = ONE
    
    return ret

In [11]:
def replaceBIOFormat(data, currbTag, curriTag, curroTag, replbTag, repliTag, reploTag):
    ret = ZERO
    
    if (data is not None and currbTag is not None and 
        curriTag is not None and curroTag is not None and 
        replbTag is not None and repliTag is not None and 
        reploTag is not None):
        
        Logger.printInfo(replacingBIOFormat.format(currbTag, replbTag, curriTag, repliTag, curroTag, reploTag))
        
        data.data.loc[data.data[data.labelColumn] == currbTag, data.labelColumn] = replbTag
        data.data.loc[data.data[data.labelColumn] == curriTag, data.labelColumn] = repliTag
        data.data.loc[data.data[data.labelColumn] == curroTag, data.labelColumn] = reploTag
        
        Logger.printInfo(replacedBIOFormat)
        
    else:
        Logger.printWarning(replacingBIOFormatNotPossible)
        ret = ONE
        
    return ret